In [ ]:
#I am using this notebook as practice to follow along with evolving hockeys tutorial
#on how they calculate their expected goal value. link to the tutorial is here
#https://evolving-hockey.com/blog/a-new-expected-goals-model-for-predicting-goals-in-the-nhl/

In [73]:
import pandas as pd
import numpy as np

In [74]:
df_import = pd.read_csv('data/washington_sample.csv')

In [75]:
filter_list = ['FAC', 'HIT', 'MISS', 'SHOT', 'BLOCK', 'GIVE', 'TAKE', 'GOAL']
df_event_fill = df_import[df_import['event_type'].isin(filter_list)]

In [8]:
df_event_fill.head()

,season,game_id,game_date,session,event_index,game_period,game_seconds,clock_time,event_type,event_description,...,game_strength_state,home_zone,pbp_distance,event_distance,event_angle,home_zonestart,face_index,pen_index,shift_index,pred_goal
0,20202021,2020020359,2021-03-03,R,7,1,0,20:00,FAC,WSH won Neu. Zone - WSH #26 DOWD vs BOS #52 KU...,...,5v5,Neu,NaN,89.00,0.00,2.0,1,0,2,NaN
1,20202021,2020020359,2021-03-03,R,8,1,24,19:36,HIT,"BOS #10 BJORK HIT WSH #3 JENSEN, Off. Zone",...,5v5,Off,NaN,30.59,78.69,NaN,1,0,2,NaN
2,20202021,2020020359,2021-03-03,R,10,1,33,19:27,HIT,"BOS #75 CLIFTON HIT WSH #26 DOWD, Def. Zone",...,5v5,Def,NaN,21.84,74.05,NaN,1,0,3,NaN
3,20202021,2020020359,2021-03-03,R,15,1,79,18:41,MISS,"WSH #8 OVECHKIN, Slap, Wide of Net, Off. Zone,...",...,5v5,Def,32.0,32.28,16.19,2.0,1,0,7,0.045674
4,20202021,2020020359,2021-03-03,R,18,1,101,18:19,HIT,"WSH #4 DILLON HIT BOS #63 MARCHAND, Def. Zone",...,5v5,Off,NaN,59.03,38.81,NaN,1,0,9,NaN


In [76]:
df = df_event_fill.copy()

In [77]:
#This page was helpful https://stackoverflow.com/questions/39109045/numpy-where-with-multiple-conditions
#Conditions for extracting the zone
col = 'event_description'
conditions = [df[col].str.contains('Off. Zone', na=False), df[col].str.contains('Def. Zone', na=False), 
              df[col].str.contains('Neu. Zone', na=False)]
choices = ['Off', 'Def','Neu']
fenwick = ['SHOT', 'MISS', 'GOAL']
corsi = ['SHOT', 'MISS', 'GOAL', 'BLOCK']
correction = ['Tip-In', 'Wrap-around', 'Deflected']

In [78]:
df['event_distance'] = ((89 - abs(df['coords_x']))**2 + df['coords_y']**2)**(1/2)
df['event_angle'] = abs(np.arctan(df['coords_y'] / (89 - abs(df['coords_x']))) * (180 / np.pi))
df['pre_event_zone'] = np.select(conditions, choices, default=np.nan)
df['pre_event_zone_2'] = np.where((df.pre_event_zone == 'Def') & 
                                  (df.event_type == 'BLOCK'), 
                                  'Off',df.pre_event_zone)
df['pre_event_zone_3'] = np.where((df.event_type.isin(fenwick)) & 
                                  (df.event_zone == 'Def') & 
                                  (df.pbp_distance <= 64),
                                  'Off',df.pre_event_zone)
df['event_distance_long'] = np.where((df.event_type.isin(fenwick)) & 
                                     (df.pbp_distance > 89) &
                                     (df.coords_x < 0) &
                                     (~df.event_detail.isin(correction)) &~
                                     ((df.pbp_distance > 89) & (df.event_zone == 'Off')),
                                     (((abs(df.coords_x)+89)**2) + df.coords_y**2)**(1/2)
                                     ,df.event_distance)
df['event_distance_long_2'] = np.where((df.event_type.isin(fenwick)) & 
                                     (df.pbp_distance > 89) &
                                     (df.coords_x > 0) &
                                     (~df.event_detail.isin(correction)) &~
                                     ((df.pbp_distance > 89) & (df.event_zone == 'Off')),
                                     (((df.coords_x+89)**2) + df.coords_y**2)**(1/2)
                                     ,df.event_distance)
df['event_angle_long'] = np.where((df.event_type.isin(fenwick)) & 
                                     (df.pbp_distance > 89) &
                                     (df.coords_x < 0) &
                                     (~df.event_detail.isin(correction)) &~
                                     ((df.pbp_distance > 89) & (df.event_zone == 'Off')),
                                     abs(np.arctan(df['coords_y']/(abs(df['coords_x'])+89))*(180 / np.pi))
                                     ,df.event_angle)
df['event_angle_long_2'] = np.where((df.event_type.isin(fenwick)) & 
                                     (df.pbp_distance > 89) &
                                     (df.coords_x > 0) &
                                     (~df.event_detail.isin(correction)) &~
                                     ((df.pbp_distance > 89) & (df.event_zone == 'Off')),
                                     abs(np.arctan(df['coords_y']/(df['coords_x']+89))*(180 / np.pi))
                                     ,df.event_angle)
df['event_zone_long'] = np.where((df.event_type.isin(fenwick)) & 
                                     (df.event_zone == 'Def') &
                                     (df.pbp_distance <= 64),
                                     'Off',
                                     df.event_zone)
df['game_strength_state'] = np.where((df.event_description.str.contains('penalty shot', na=False)) &
                                     (df.event_team == df.home_team),
                                     'Ev1',
                                     df.game_strength_state)
df['game_strength_state'] = np.where((df.event_description.str.contains('penalty shot', na=False)) &
                                     (df.event_team == df.away_team),
                                     '1vE',
                                     df.game_strength_state)
df['home_skaters'] = np.where((df.event_description.str.contains('penalty shot', na=False)) &
                                     (df.event_team == df.home_team),
                                     1,
                                     df.home_skaters)
df['home_skaters'] = np.where((df.event_description.str.contains('penalty shot', na=False)) &
                                     (df.event_team == df.away_team),
                                     0,
                                     df.home_skaters)
df['away_skaters'] = np.where((df.event_description.str.contains('penalty shot', na=False)) &
                                     (df.event_team == df.home_team),
                                     0,
                                     df.away_skaters)
df['away_skaters'] = np.where((df.event_description.str.contains('penalty shot', na=False)) &
                                     (df.event_team == df.away_team),
                                     1,
                                     df.away_skaters)

In [79]:
event_interest = ["FAC", "GOAL", "BLOCK", "SHOT", "MISS", "HIT", "TAKE", "GIVE"]
filter_1 = df.event_type.isin(event_interest)
filter_2 = df.game_period < 5
filter_3 = ~df.event_description.str.contains('penalty shot', na=False)
filter_4 = df.coords_x.isna()
filter_5 = df.coords_y.isna()
filter_6 = ~((df.coords_x == 0) &
             (df.coords_y == 0) &
             (df.event_type.isin(corsi)) &
             ((df.pbp_distance != 90) & (df.event_type.isin(fenwick))))

In [93]:
#df_2 = df[filter_1 & filter_2 & filter_3 & filter_4 & filter_5 & filter_6].copy()
df_2 = df[filter_1 & filter_2 & filter_3 & filter_6].copy()

In [96]:
df_2.dropna(subset=['coords_x','coords_y'],inplace=True)

In [97]:
df_2.shape[0]

234

In [82]:
df.shape[0]

239

In [58]:
df.columns

Index(['season', 'game_id', 'game_date', 'session', 'event_index',
       'game_period', 'game_seconds', 'clock_time', 'event_type',
       'event_description', 'event_detail', 'event_zone', 'event_team',
       'event_player_1', 'event_player_2', 'event_player_3', 'event_length',
       'coords_x', 'coords_y', 'num_on', 'num_off', 'players_on',
       'players_off', 'home_on_1', 'home_on_2', 'home_on_3', 'home_on_4',
       'home_on_5', 'home_on_6', 'home_on_7', 'away_on_1', 'away_on_2',
       'away_on_3', 'away_on_4', 'away_on_5', 'away_on_6', 'away_on_7',
       'home_goalie', 'away_goalie', 'home_team', 'away_team', 'home_skaters',
       'away_skaters', 'home_score', 'away_score', 'game_score_state',
       'game_strength_state', 'home_zone', 'pbp_distance', 'event_distance',
       'event_angle', 'home_zonestart', 'face_index', 'pen_index',
       'shift_index', 'pred_goal', 'pre_event_zone', 'pre_event_zone_2',
       'pre_event_zone_3', 'event_distance_long', 'test',
       'e

In [65]:
df.game_period.head()

0    1
1    1
2    1
3    1
4    1
Name: game_period, dtype: int64